In [1]:
#read datafolder from Google Drive
from google.colab import drive
drive.mount('/content/drive')
data_folder = '/content/drive/My Drive/pg2k18/sem2/smai/smai_proj/reuters21578/'


sgml_number_of_files = 21
sgml_file_name_template = 'reut2-NNN.sgm'

# Category files
category_files = {
    'to_': ('Topics', 'all-topics-strings.lc.txt'),
    'pl_': ('Places', 'all-places-strings.lc.txt'),
    'pe_': ('People', 'all-people-strings.lc.txt'),
    'or_': ('Organizations', 'all-orgs-strings.lc.txt'),
    'ex_': ('Exchanges', 'all-exchanges-strings.lc.txt')
}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [0]:
# Read all categories
category_data = []
category_dictionary={'Topics':[],'Places':[],'People':[],'Organizations':[],'Exchanges':[]}
for category_prefix in category_files.keys():
    with open(data_folder + category_files[category_prefix][1], 'r') as file:
        for category in file.readlines():
            category_data.append([category_prefix + category.strip().lower(), 
                                  category_files[category_prefix][0]])

# Create category dataframe
for i in category_data:
    category_dictionary[i[1]].append(i[0].split('_')[1])
news_categories = pd.DataFrame(data=category_data)


In [4]:
import re
import xml.sax.saxutils as saxutils
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
lemmatizer = WordNetLemmatizer()

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;\\n]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def cleanUpSentence(r):#, stop_words = None#
    r = r.lower().replace("<br />", " ")
    r = REPLACE_BY_SPACE_RE.sub(' ', r)
    r = BAD_SYMBOLS_RE.sub('', r)

    r = ' '.join(word for word in r.split() if word not in STOPWORDS)

    words = word_tokenize(r)

    for w in words:
        w = lemmatizer.lemmatize(w)

    return r

In [0]:
# Parse SGML files
def strip_tags(text):
    return re.sub('<[^<]+?>', '', text).strip()

def unescape(text):
    return saxutils.unescape(text)
  
def makeDict(filename, document_X, document_Y):
  with open(filename, 'rb') as file:

    content = BeautifulSoup(file.read().lower(),'html.parser')

    for newsline in content('reuters'):
      document_categories = []

      document_id = newsline['newid']



      document_body = strip_tags(str(newsline('text')[0].body)).replace('reuter\n&#3;', '')
      if document_body == 'None':
        continue

      doc_categories = strip_tags(str(newsline('topics')[0].body))
      doc_categories = unescape(doc_categories)

      document_body = unescape(document_body)

      topics = newsline.topics.contents
      places = newsline.places.contents
      people = newsline.people.contents
      orgs = newsline.orgs.contents
      exchanges = newsline.exchanges.contents

      for topic in topics:
          document_categories.append('to_' + strip_tags(str(topic)))

      for place in places:
          document_categories.append('pl_' + strip_tags(str(place)))

      for person in people:
          document_categories.append('pe_' + strip_tags(str(person)))

      for org in orgs:
          document_categories.append('or_' + strip_tags(str(org)))

      for exchange in exchanges:
          document_categories.append('ex_' + strip_tags(str(exchange)))

      document_X[document_id] = document_body
      document_Y[document_id] = document_categories

def readFiles(test_data = False):
  document_X = {}
  document_Y = {}
  if test_data == True:
    file_name = sgml_file_name_template.replace('NNN', '021')
    filename = data_folder + file_name
    makeDict(filename, document_X, document_Y)
  else:
    for i in range(sgml_number_of_files):
      if i < 10:
        seq = '00' + str(i)
      else:
        seq = '0' + str(i)

      file_name = sgml_file_name_template.replace('NNN', seq)
      print('Reading file: %s' % file_name)
      filename = data_folder + file_name
      makeDict(filename, document_X, document_Y)
  return document_X, document_Y



In [7]:
document_X, document_Y = readFiles()

Reading file: reut2-000.sgm
Reading file: reut2-001.sgm
Reading file: reut2-002.sgm
Reading file: reut2-003.sgm
Reading file: reut2-004.sgm
Reading file: reut2-005.sgm
Reading file: reut2-006.sgm
Reading file: reut2-007.sgm
Reading file: reut2-008.sgm
Reading file: reut2-009.sgm
Reading file: reut2-010.sgm
Reading file: reut2-011.sgm
Reading file: reut2-012.sgm
Reading file: reut2-013.sgm
Reading file: reut2-014.sgm
Reading file: reut2-015.sgm
Reading file: reut2-016.sgm
Reading file: reut2-017.sgm
Reading file: reut2-018.sgm
Reading file: reut2-019.sgm
Reading file: reut2-020.sgm


In [8]:
# data preprocessing
import numpy as np

nltk.download('punkt')
nltk.download('wordnet')

def create_x_matrix(document_X):
    totalX = []
    for i, doc in document_X.items():
        totalX.append(cleanUpSentence(doc))
    max_vocab_size = 200
    input_tokenizer = Tokenizer(200)
    input_tokenizer.fit_on_texts(totalX)
    encoded_docs = input_tokenizer.texts_to_matrix(totalX, mode='count')
    return totalX,encoded_docs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
totalX,encoded_docs=create_x_matrix(document_X)

In [0]:
#Create one-hot encode
words_in_body={}

for i in range(len(totalX)):
    words=totalX[i].split(' ')
    words_in_body[i]=words    

one_hot_label=[]
for key,v in words_in_body.items():
    dict_temp={'Topics':0,'Places':0,'People':0,'Exchanges':0,'Organizations':0}
    for i in v:
        if i in category_dictionary['Topics']:
            dict_temp['Topics']+=1
        if i in category_dictionary['Places']:
            dict_temp['Places']+=1
        if i in category_dictionary['People']:
            dict_temp['People']+=1
        if i in category_dictionary['Exchanges']:
            dict_temp['Exchanges']+=1
        if i in category_dictionary['Organizations']:
            dict_temp['Organizations']+=1
            
    one_hot_label.append(dict_temp)
    

one_hot_label_list = []
for i in one_hot_label:

    one_hot_label_list.append(list(i.values()))
    

In [0]:
def permissible(x, y):
  if (abs(x-y)) < 1:
    return True
  else:
    return False

def accuracy(predicted,actual,length):
    tp=0
   
    for one_doc_idx in range(length):
        if permissible(predicted[one_doc_idx][0],actual[one_doc_idx][0]) and permissible(predicted[one_doc_idx][1],actual[one_doc_idx][1])\
          and permissible(predicted[one_doc_idx][2],actual[one_doc_idx][2]) and permissible(predicted[one_doc_idx][3],actual[one_doc_idx][3]):
            tp+=1
    return tp/float(length)
  

In [0]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
X = np.array(encoded_docs)
y = np.array(one_hot_label_list)
print (y)

[[7 3 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 0]
 ...
 [0 0 0 0 0]
 [5 0 0 1 0]
 [1 0 0 0 0]]


In [0]:
clf = DecisionTreeClassifier(random_state=0).fit(X, y)

In [16]:
#Prediction and Calculating Accuracy
preds = clf.predict(X)

print ("%f"%accuracy(preds.tolist(), one_hot_label_list, len(one_hot_label_list)))

0.999300
